* 페이스북에서 만든 FastText
* Word2vec 확장이지만, 차이점은 Word2vec은 단어를 쪼갤 수 없는 단위로 생각한다면, FastText는 하나의 단어 안에도 여러 단어들이 존재하는 것으로 간주
* 즉, 내부단어(subword)를 고려하여 학습

### 1. 내부단어(subword) 학습

* 각 단어는 글자단위 n-gram의 구성으로 취급
* n을 몇으로 결정할지에 따라 단어들이 얼마나 분리되는지 결정됨
* 예를 들어 n을 3으로 하는 경우, apple은 app, ppl, ple로 분리하고 이들을 벡터로 만들게 됨

* 여기에 추가적으로 하나를 더 벡터화하는데, 기존 단어에 <, 와>를 붙인 토큰임
* 다시 말해 n = 3인 경우, FastText는 단어 apple에 대해서 다음의 6개의 토큰을 벡터화함

* 실제 사용할 때는 n의 최소값과 최대값으로 범위를 설정할 수 있는데, 기본값으로 각각 3과 6으로 설정되어져 있음. 다시 말해 최소값 = 3, 최대값 = 6인 경우, 단어 apple에 대해서 FastText는 아래 내부 단어들을 벡터화함

* 여기서 내부 단어들을 벡터화한다는 의미는 위의 단어들에 대해 Word2Vec을 수행한다는 의미
* 내부 단어들의 벡터값을 얻었다면, 단어 apple의 벡터값은 저 위 벡터값들의 총 합으로 구성됨

### 2. 모르는 단어(Out Of Vocabulary, OOV)에 대한 대응

* FastText의 인공신경망을 학습한 후에는 데이터 셋의 모든 단어의 각 n-gram에 대해 워드 임베딩이 됨
* 이렇게 되면 장점은 데이터셋만 충분하다면 위와 같은 내부 단어(subword)를 통해 모르는 단어(OOV)에 대해서도 다른 단어와의 유사도를 계산할 수 있다는 것임
* 예를 들어, FastText에서 birthplace란 단어를 학습하지 않은 상태이지만, 다른 단어에서 birth와 place라는 내부단어가 있다면 FastText는 birthplace의 벡터를 얻을 수 있음

----
* 모르는 단어에 제대로 대체할 수 없었던 Word2Vec, GloVe와는 다른 점임

### 3. 단어 집합 내 빈도수가 적었던 단어(Rare Word)에 대한 대응
* Word2Vec의 경우 등장빈도수가 적은 단어(rare word)에 대해 임베딩 정확도가 높지 않다는 단점이 있음
* 참고로 할 수 있는 사례가 적다보니 정확하게 임베딩이 되지 않는 것임
* 그러나, FastText의 경우 빈도수가 적은 단어라도 그 단어의 n-gram이 다른 단어의 n-gram과 겹치게 되면 Word2Vec에 비해 비교적 높은 임베딩 벡터값을 얻음
* FastText가 노이즈가 많은 코퍼스에서 강점을 가진 것 또는 이 때문임. 즉, 오타가 섞인 단어는 당연히 등장 빈도수가 매우 적으므로 일종의 희귀 단어가 되는데 Word2Vec에서는 오타가 섞인 단어는 임베딩이 제대로 되지 않지만 FastText는 이에 대해서도 일정 수준의 성능을 보임
* 예를 들어 단어 apple과 오타로 p를 한 번 더 입력한 appple의 경우에는 실제로 많은 개수의 동일한 n-gram을 가짐

### 4. Word2Vec vs. FastText 실습 비교

#### 1) Word2Vec

* 앞에서 만든 모델 이용

In [4]:
## 모델 로드
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("eng_w2v")

In [5]:
model.most_similar("electrofishing")  

KeyError: "Key 'electrofishing' not present"

* 존재하지 않는 단어를 에러 발생

#### 2) FastText
* 전처리만 이용하고 FastText 학습 코드로 변경하여 실행

In [6]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [8]:
from gensim.models import FastText

model = FastText(result, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [9]:
model.wv.most_similar("electrofishing")

[('electrolux', 0.8636071681976318),
 ('electrolyte', 0.8578426837921143),
 ('electroshock', 0.8490543365478516),
 ('electroencephalogram', 0.8438616394996643),
 ('electro', 0.8402094841003418),
 ('electrochemical', 0.8326069712638855),
 ('airbus', 0.8204637765884399),
 ('electrogram', 0.8197062015533447),
 ('electric', 0.81430584192276),
 ('electronic', 0.811857283115387)]

### 5. 한국어에서의 FastText
* 한국어의 경우에도 OOV 문제 해결을 위해 FastText 적용 시도

#### 1) 음절 단위
* 음절단위의 임베딩의 경우, n=3일때 '자연어처리'라는 단어에 대해 n-gram을 만들면 다음과 같음

#### 2) 자모단위
* 자모단위(초성, 중성, 종성 단위)로 임베딩
* 자모단위로 하게 되면 오타나 노이즈 측면에서 더 강한 임베딩을 기대할 수 있음
* 예를 들어,  ‘자연어처리’라는 단어는 아래와 같이 분리됨

분리된 결과 : ㅈ ㅏ _ ㅇ ㅕ ㄴ ㅇ ㅓ _ ㅊ ㅓ _ ㄹ ㅣ _

이렇게 분리된 결과를 n=3의 n-gram을 적용하면 임베딩 결과는 다음과 같음

In [ ]:
### 1. 코퍼스 만들기 ===>> colab